# Transform spike rates to behaviour using RNN

## Setup

### Environment Setup

Configure the local or Google Colab environments.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import os
import sys
try:
    # Only on works on Google Colab
    from google.colab import files
    %tensorflow_version 2.x
    os.chdir('..')
    
    # Configure kaggle if necessary
    if not (Path.home() / '.kaggle').is_dir():
        uploaded = files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
        if 'kaggle.json' in uploaded.keys():
            !mkdir -p ~/.kaggle
            !mv kaggle.json ~/.kaggle/
            !chmod 600 ~/.kaggle/kaggle.json
    
    !pip install git+https://github.com/SachsLab/indl.git
    
    if Path.cwd().stem == 'MonkeyPFCSaccadeStudies':
        os.chdir(Path.cwd().parent)
    
    if not (Path.cwd() / 'MonkeyPFCSaccadeStudies').is_dir():
        !git clone --single-branch --recursive https://github.com/SachsLab/MonkeyPFCSaccadeStudies.git
        sys.path.append(str(Path.cwd() / 'MonkeyPFCSaccadeStudies'))
    
    os.chdir('MonkeyPFCSaccadeStudies')
        
    !pip install -q kaggle
    
    # Latest version of SKLearn
    !pip install -U scikit-learn
    
    IN_COLAB = True
    
except ModuleNotFoundError:    
    # chdir to MonkeyPFCSaccadeStudies
    if Path.cwd().stem == 'Analysis':
        os.chdir(Path.cwd().parent.parent)
        
    # Add indl repository to path.
    # Eventually this should already be pip installed, but it's still under heavy development so this is easier for now.
    check_dir = Path.cwd()
    while not (check_dir / 'Tools').is_dir():
        check_dir = check_dir / '..'
    indl_path = check_dir / 'Tools' / 'Neurophys' / 'indl'
    sys.path.append(str(indl_path))
    
    # Make sure the kaggle executable is on the PATH
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')
    
    IN_COLAB = False

# Try to clear any logs from previous runs
if (Path.cwd() / 'logs').is_dir():
    import shutil
    try:
        shutil.rmtree(str(Path.cwd() / 'logs'))
    except PermissionError:
        print("Unable to remove logs directory.")

In [ ]:
# Additional imports
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
from indl.display import turbo_cmap

In [ ]:
plt.rcParams.update({
    'axes.titlesize': 24,
    'axes.labelsize': 20,
    'lines.linewidth': 2,
    'lines.markersize': 5,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 18,
    'figure.figsize': (6.4, 6.4)
})

### Download Data (if necessary)

In [ ]:
if IN_COLAB:
    data_path = Path.cwd() / 'data' / 'monkey_pfc' / 'converted'
else:
    data_path = Path.cwd() / 'StudyLocationRule' / 'Data' / 'Preprocessed'

if not (data_path).is_dir():
    !kaggle datasets download --unzip --path {str(data_path)} cboulay/macaque-8a-spikes-rates-and-saccades
    print("Finished downloading and extracting data.")
else:
    print("Data directory found. Skipping download.")

### (Prepare to) Load Data

We will use a custom function `load_macaque_pfc` to load the data into memory.

There are 4 different strings to be passed to the import `x_chunk` argument:
* 'analogsignals' - if present. Returns 1 kHz LFPs
* 'gaze'          - Returns 2-channel gaze data.
* 'spikerates'    - Returns smoothed spikerates
* 'spiketrains'

The `y_type` argument can be
* 'pair and choice' - returns Y as np.array of (target_pair, choice_within_pair)
* 'encoded input' - returns Y as np.array of shape (n_samples, 10) (explained below)
* 'replace with column name' - returns Y as a vector of per-trial values. e.g., 'sacClass'

The actual data we load depends on the particular analysis below.

In [ ]:
from misc.misc import sess_infos, load_macaque_pfc, dec_from_enc

load_kwargs = {
    'valid_outcomes': (0,),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, 1.45),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}

## Decoding trial class (0:7) from spike rates

### Load Data

Load the per-trial spikerates and saccadeClass (0:7)

In [ ]:
test_sess_ix = 1
sess_info = sess_infos[test_sess_ix]
sess_id = sess_info['exp_code']
print(f"\nImporting session {sess_id}")
X_rates, Y_class, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs)
Y_class = tf.keras.utils.to_categorical(Y_class, num_classes=8)

### Create Model

Our model objective is to transform timeseries segments into probabilities of each class.

Let's start with a function to make the model:

In [ ]:
from indl.model import parts
from indl.model.helper import check_inputs
from indl.regularizers import KernelLengthRegularizer
from tensorflow.keras import layers

@check_inputs
def make_model(
    _input,
    filt=8,
    kernLength=25,
    ds_rate=10,
    n_rnn=64,
    n_rnn2=64,
    dropoutRate=0.25,
    activation='relu',
    l1_reg=0.000, l2_reg=0.000,
    norm_rate=0.25,
    latent_dim=16,
    return_model=True
):
    _y = _input
    
    input_shape = _input.shape.as_list()
    # The Conv layers are insensitive to the number of samples in the time dimension.
    # To make it possible for this trained model to be applied to segments of different
    # durations, we need to explicitly state that we don't care about the number of samples.
    # input_shape[2] = -1  # Comment out during debug
    # _y = layers.Reshape(input_shape[1:])(_input)  # Note that Reshape ignores the batch dimension.

    # RNN
    if len(input_shape) < 4:
        input_shape = input_shape + [1]
    # The Conv layers are insensitive to the number of samples in the time dimension.
    # To make it possible for this trained model to be applied to segments of different
    # durations, we need to explicitly state that we don't care about the number of samples.
    _y = layers.Reshape(input_shape[1:])(_y)
    _y = tf.keras.layers.Conv2D(filt, (1, kernLength), padding='valid', data_format=None,
                                dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform',
                                bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
                                activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(_y)
    _y = tf.keras.layers.BatchNormalization()(_y)
    _y = tf.keras.layers.DepthwiseConv2D((_y.shape.as_list()[1], 1), padding='valid',
                                      depth_multiplier=1, data_format=None, dilation_rate=(1, 1),
                                      activation=None, use_bias=True, depthwise_initializer='glorot_uniform',
                                      bias_initializer='zeros', depthwise_regularizer=None,
                                      bias_regularizer=None, activity_regularizer=None,
                                      depthwise_constraint=None, bias_constraint=None)(_y)
    _y = tf.keras.layers.BatchNormalization()(_y)
    _y = tf.keras.layers.Activation(activation)(_y)
    _y = tf.keras.layers.AveragePooling2D(pool_size=(1, ds_rate))(_y)
    _y = tf.keras.layers.Dropout(dropoutRate)(_y)
    _y = layers.Reshape(_y.shape.as_list()[2:])(_y)
    _y = tf.keras.layers.LSTM(n_rnn,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                              recurrent_regularizer=tf.keras.regularizers.l2(l2_reg),
                              return_sequences=n_rnn2 > 0,
                              stateful=False,
                              name='rnn1')(_y)
    _y = tf.keras.layers.Activation(activation)(_y)
    _y = tf.keras.layers.BatchNormalization()(_y)
    _y = tf.keras.layers.Dropout(dropoutRate)(_y)
    
    
    if n_rnn2 > 0:
        
        _y = tf.keras.layers.LSTM(n_rnn2,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                              recurrent_regularizer=tf.keras.regularizers.l2(l2_reg),
                              return_sequences=False,
                              stateful=False,
                              name='rnn2')(_y)
        _y = tf.keras.layers.Activation(activation)(_y)
        _y = tf.keras.layers.BatchNormalization()(_y)
        _y = tf.keras.layers.Dropout(dropoutRate)(_y)
    
    # Dense
    _y = parts.Bottleneck(_y, latent_dim=latent_dim, activation=activation)
    
    # Classify
    _y = parts.Classify(_y, n_classes=8, norm_rate=norm_rate)
    

    if return_model is False:
        return _y
    else:
        return tf.keras.models.Model(inputs=_input, outputs=_y)

In [ ]:
LABEL_SMOOTHING = 0.2
model_kwargs = dict(
    filt=8,
    kernLength=25,
    ds_rate=9,
    n_rnn=64,
    n_rnn2=64,
    dropoutRate=0.40,
    activation='relu',
    l1_reg=0.0000, l2_reg=0.001,
    norm_rate=0.25,
    latent_dim=32
)

tf.keras.backend.clear_session()

randseed = 12345
random.seed(randseed)
np.random.seed(randseed)
tf.random.set_seed(randseed)

input_spec = tf.data.Dataset.from_tensor_slices(X_rates).batch(16, drop_remainder=True).element_spec
model = make_model(
    input_spec,
    **model_kwargs
)
optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])
model.summary()
if False:
    tf.keras.utils.plot_model(
        model,
        to_file='model.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB',
        expand_nested=False,
        dpi=96
    )

### Training Loop

In [ ]:
from sklearn.model_selection import StratifiedKFold

N_SPLITS = 10
BATCH_SIZE = 16
EPOCHS = 180


def get_hists_acc(sess_id, verbose=1):
    print(f"Processing session {sess_id}...")
    X_rates, Y_class, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs)
    
    splitter = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=0)
    split_ix = 0
    histories = []
    per_fold_eval = []
    per_fold_true = []

    for trn, vld in splitter.split(X_rates, Y_class):
        print(f"\tSplit {split_ix + 1} of {N_SPLITS}")
        _y = tf.keras.utils.to_categorical(Y_class, num_classes=8)
        
        ds_train = tf.data.Dataset.from_tensor_slices((X_rates[trn], _y[trn]))
        ds_valid = tf.data.Dataset.from_tensor_slices((X_rates[vld], _y[vld]))

        # cast data types to GPU-friendly types.
        ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))
        ds_valid = ds_valid.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))

        # TODO: augmentations (random slicing?)

        ds_train = ds_train.shuffle(len(trn) + 1)
        ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
        ds_valid = ds_valid.batch(BATCH_SIZE, drop_remainder=False)

        tf.keras.backend.clear_session()
        
        randseed = 12345
        random.seed(randseed)
        np.random.seed(randseed)
        tf.random.set_seed(randseed)
        
        model = make_model(
            ds_train.element_spec[0],
            **model_kwargs
        )
        optim = tf.keras.optimizers.Nadam(learning_rate=0.001)
        loss_obj = tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING)
        model.compile(optimizer=optim, loss=loss_obj, metrics=['accuracy'])
        
        best_model_path = f'r2c_lstm_{sess_id}_split{split_ix}.h5'
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=best_model_path,
                # Path where to save the model
                # The two parameters below mean that we will overwrite
                # the current checkpoint if and only if
                # the `val_loss` score has improved.
                save_best_only=True,
                monitor='val_accuracy',
                verbose=verbose)
        ]

        hist = model.fit(x=ds_train, epochs=EPOCHS,
                         verbose=verbose,
                         validation_data=ds_valid,
                         callbacks=callbacks)
        # tf.keras.models.save_model(model, 'model.h5')
        histories.append(hist.history)
        
        model = tf.keras.models.load_model(best_model_path)
        per_fold_eval.append(model(X_rates[vld]).numpy())
        per_fold_true.append(Y_class[vld])
        
        split_ix += 1
        
    # Combine histories into one dictionary.
    history = {}
    for h in histories:
        for k,v in h.items():
            if k not in history:
                history[k] = v
            else:
                history[k].append(np.nan)
                history[k].extend(v)
                
    pred_y = np.concatenate([np.argmax(_, axis=1) for _ in per_fold_eval])
    true_y = np.concatenate(per_fold_true).flatten()
    accuracy = 100 * np.sum(pred_y == true_y) / len(pred_y)
    print(f"Session {sess_id} overall accuracy: {accuracy}%")
    
    return history, accuracy
        

In [ ]:
from indl.metrics import quickplot_history

history, accuracy = get_hists_acc(sess_infos[1]['exp_code'], verbose=2)
quickplot_history(history)

### Training and evaluation - per session

Note that using verbose=0 will suppress most of the log information. Progress indicators might only be updated once every few minutes, depending on the number of epochs and the amount of time it takes to process a session.

In [ ]:
hists = []
accs = []
for sess_info in sess_infos:
    _hist, _acc = get_hists_acc(sess_info['exp_code'], verbose=0)
    hists.append(_hist)
    accs.append(_acc)
    
print(accs)

### Finding the Best Model (Manual Hyperparameter Optimization)

In [ ]:
from datetime import datetime

input_spec = tf.data.Dataset.from_tensor_slices(X_rates).batch(16, drop_remainder=True).element_spec

dsrate = 9
rnn1s = [32, 64, 128, 256]
rnn2s = [32, 64, 128, 256]
denses = [0, 8, 16, 32, 64, 128]
meanacc = 0
best_rnn1 = 0
best_rnn2 = 0
best_dense = 0
best_acc = []
best_model = None
count=1
for rnn1 in rnn1s:
    for rnn2 in rnn2s:
        for dense in denses:
            f = open("C:/Users/alire/OneDrive/rnnLog.txt", "a")
            f.write(f"\n\nLoop # {count} of {len(rnn1s)*len(rnn2s)*len(denses)}\n")
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            f.write(str(current_time))
            print(f"Loop # {count} of 96")
            print(current_time)
            count+=1
            model_kwargs = dict(
                filt=8,
                kernLength=25,
                ds_rate=9,
                n_rnn=rnn1,
                n_rnn2=rnn2,
                dropoutRate=0.40,
                activation='relu',
                l1_reg=0.0000, l2_reg=0.001,
                norm_rate=0.25,
                latent_dim=dense
            )
            model = make_model(
                input_spec,
                **model_kwargs
            )
            hists = []
            accs = []
            for sess_info in sess_infos:
                _hist, _acc = get_hists_acc(sess_info['exp_code'], verbose=0)
                hists.append(_hist)
                accs.append(_acc)
            if np.mean(accs) > meanacc:
                meanacc = np.mean(accs)
                f.write(f"\nAverage Accuracy Improved to: {meanacc}\n")
                f.write(str(accs))
                print(f"Average Accuracy Improved to: {meanacc}")
                print(accs)
                f.write(f"\nRNN1: {rnn1}, RNN2: {rnn2}, Bottle: {dense}")
                print(f"RNN1: {rnn1}, RNN2: {rnn2}, Bottle: {dense}")
                best_rnn1 = rnn1
                best_rnn2 = rnn2
                best_dense = dense
                best_acc = accs
                best_model = model
                f.close()                    

f.close()
f.close()
f.close()

### Top 6 Models

All models have similiar architecture:
* Two covoloutional layers to decrease number of channels
* A dense layer to downsample timeseries
* First RNN (RNN1)
* Second RNN (RNN2)
* Dense Bottleneck (Botl)
* Dense Classifier

Model 1: RNN1 = 32(GRU), RNN2 = 32(GRU), Botl = 64

Model 2: RNN1 = 32(LSTM), RNN2 = 64(LSTM), Botl = 64

Model 3: RNN1 = 32(LSTM), RNN2 = 128(LSTM), Botl = 16

Model 4: RNN1 = 32(LSTM), RNN2 = 128(LSTM), Botl = 128

Model 5: RNN1 = 32(LSTM), RNN2 = 256(LSTM), Botl = 16

Model 6: RNN1 = 64(LSTM), RNN2 = 128(LSTM), Botl = 32

In [ ]:
labels = ['Model1', 'Model2', 'Model3', 'Model4', 'Model5', 'Model6']
data = np.array([[63.75,           73.125,             71.25,               70.,            71.875,             73.75],
                [88.42, 88.42105263157895,  86.3157894736842, 86.66666666666667, 87.71929824561404,  88.0701754385965],
                [80.71, 75.64575645756457, 77.49077490774907, 76.75276752767527, 77.12177121771218, 79.33579335793358],
                [74.,   77.61732851985559, 78.33935018050542, 80.50541516245487, 79.06137184115524, 79.06137184115524],
                [67.1875,         70.3125,          66.40625,          66.40625,           70.3125,          64.84375],
                [83.33,            71.875, 77.08333333333333,            78.125, 77.08333333333333,            78.125],
                [85.93,           79.6875,           85.9375,            84.375,            84.375,            84.375],
                [80.,   81.17647058823529, 81.17647058823529,  82.3529411764706, 81.17647058823529,  82.3529411764706]])
                     
sess_names = [_['exp_code'].split('_')[2] for _ in sess_infos]

In [ ]:
fig1, ax1 = plt.subplots(figsize=(6, 4))

plt.axhline(70, color='k', linestyle='--')

ax1.boxplot(data, showfliers=False, medianprops=dict(color='k'))
ax1.set_xticklabels(labels, fontsize=12)
plt.ylabel('Accuracy (%)')
plt.xlabel('Model')

for row_ix, m in zip(range(data.shape[0]), ['o', '^', '1', 's', '+', 'x', 'd', '*']):
    plt.scatter([1, 2, 3, 4, 5, 6], data[row_ix, :], marker=m)

plt.tight_layout()
fig1.savefig('classif_acc.png')